### Processing Images with OpenCV 3

#### Converting between different color spaces

- three color spaces are prevalent in modern day computer
vision: gray, BGR, and Hue, Saturation, Value (HSV).

- Gray is a color space that effectively eliminates color information translating
to shades of gray: this color space is extremely useful for intermediate
processing, such as face detection.

- BGR is the blue-green-red color space, in which each pixel is a three-element
array, each value representing the blue, green, and red colors: web developers
would be familiar with a similar definition of colors, except the order of colors
is RGB.

- In HSV, hue is a color tone, saturation is the intensity of a color, and value
represents its darkness (or brightness at the opposite end of the spectrum).

- [0 255 255] value (no blue, full green, and full red) produces the yellow
color.


#### The Fourier Transform

- Much of the processing you apply to images and videos in OpenCV involves the
concept of Fourier Transform in some capacity.

- Waveforms: useful when manipulating images, because it
allows us to identify regions in images where a signal (such as image pixels) changes
a lot, and regions where the change is less dramatic.  We can then arbitrarily mark
these regions as noise or regions of interests, background or foreground, and so on.

In an OpenCV context:
- There are a number of algorithms implemented
that enable us to process images and make sense of the data contained
in them, and these are also reimplemented in NumPy to make our
life even easier. NumPy has a Fast Fourier Transform (FFT) package,
which contains the fft2() method. This method allows us to
compute Discrete Fourier Transform (DFT) of the image.

- used for common
image processing operations, such as edge detection or line and shape detection.

Magnitude spectrum:
- It gives
a representation of the original image in terms of its changes: think of it as taking
an image and dragging all the brightest pixels to the center. Then, you gradually
work your way out to the border where all the darkest pixels have been pushed.
Immediately, you will be able to see how many light and dark pixels are contained in
your image and the percentage of their distribution

##### High pass filter
 - A high pass filter (HPF) is a filter that examines a region of an image and boosts
the intensity of certain pixels based on the difference in the intensity with the
surrounding pixels.

- Particularly effective in edge detection, where a common form of HPF called
high boost filter is used.

In [29]:
import cv2 
import numpy as np
from scipy import ndimage


"""Note that both filters sum up to 0, the reason for this is 
    explained in detail in the Edge detection section
"""

# Filter  3 by 3
kernel_3x3 = np.array([  [-1, -1, -1],
                          [-1, 8, -1],
                          [-1, -1, -1]
                                        ])
# Filter 5 by 5
kernel_5x5 = np.array([  [-1, -1, -1, -1, -1],
                          [-1, 1,  2,  1, -1],
                          [-1, 2,  4,  2, -1],
                          [-1, 1,  2,  1, -1],
                          [-1, -1,  -1,  -1, -1],
                                                ]) 
# Read the dog image
img = cv2.imread("dog.jpg", 0)    

# ndimage Multidimensional Image Processiog : Filters Fourier filters Interpolation Measurements Morphology
# convolve(): The array is convolved with the given kernel.
k3 = ndimage.convolve(img, kernel_3x3)
k5 = ndimage.convolve(img, kernel_5x5)

blurred = cv2.GaussianBlur(img, (11,11), 0)
g_hpf = img - blurred

cv2.imshow("3x3", k3)
cv2.imshow("5x5", k5)
cv2.imshow("g_hpf", g_hpf)
cv2.waitKey()
cv2.destroyAllWindows()

#### Low pass filter
- HPF boosts the intensity of a pixel, given its difference with its neighbors, a
low pass filter (LPF) will smoothen the pixel if the difference with the surrounding
pixels is lower than a certain threshold. This is used in denoising and blurring.

- Gaussian blur, is a low pass filter that attenuates the intensity of high frequency signals.

#### Edge Detection
-  Humans, can
easily recognize many object types and their pose just by seeing a backlit silhouette
or a rough sketch. Indeed, when art emphasizes edges and poses, it often seems
to convey the idea of an archetype, such as Rodin's The Thinker or Joe Shuster's
Superman. 

- OpenCV provides many edge-finding filters, including Laplacian(), Sobel(), and
Scharr().

- OpenCV also provides many blurring filters, including blur() (simple
average), medianBlur(), and GaussianBlur().



##### Edge detection with Canny
 - The Canny edge detection algorithm is quite complex but also interesting: it's a
five-step process that denoises the image with a Gaussian filter, calculates gradients,
applies non maximum suppression (NMS) on edges, a double threshold on all the
detected edges to eliminate false positives, and, lastly, analyzes all the edges and
their connection to each other to keep the real edges and discard the weak ones

In [30]:
import cv2
import numpy as np
img = cv2.imread("dog.jpg", 0)
cv2.imwrite("canny.jpg", cv2.Canny(img, 200, 300))
cv2.imshow("canny", cv2.imread("canny.jpg"))
cv2.waitKey()
cv2.destroyAllWindows()

#### Contour detection

In [ ]:
import cv2
import numpy as np

img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 255

ret, thresh = cv2.threshold(img, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
img = cv2.drawContours(color, contours, -1, (0,255,0), 2)
cv2.imshow("contours", color)
cv2.waitKey()
cv2.destroyAllWindows()

#### Contours – bounding box, minimum area rectangle, and minimum enclosing circle

In [57]:
import cv2
import numpy as np
img = cv2.pyrDown(cv2.imread("dog.jpg", cv2.IMREAD_UNCHANGED))

In [ ]:
ret, thresh = cv2.threshold(cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY) , 127, 255, cv2.THRESH_BINARY)
image, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # find bounding box coordinates
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
    # find minimum area
    rect = cv2.minAreaRect(c)
    # calculate coordinates of the minimum area rectangle
    box = cv2.boxPoints(rect)
    # normalize coordinates to integers
    box = np.int0(box)
    # draw contours
    cv2.drawContours(img, [box], 0, (0,0, 255), 3)
    # calculate center and radius of minimum enclosing circle
    (x,y),radius = cv2.minEnclosingCircle(c)
    # cast to integers
    center = (int(x),int(y))
    radius = int(radius)
    # draw the circle
    img = cv2.circle(img,center,radius,(0,255,0),2)

cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
cv2.imshow("contours", img)

#### Contours – convex contours and the Douglas-Peucker algorithm

In [ ]:
epsilon = 0.01 * cv2.arcLength(cnt, True)
approx = cv2.approxPolyDP(cnt, epsilon, True)

hull = cv2.convexHull(cnt)



#### Line and circle detection


In [68]:
import cv2
import numpy as np
img = cv2.imread('dog.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,120)
minLineLength = 20
maxLineGap = 5
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength, maxLineGap)
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    cv2.imshow("edges", edges)
    cv2.imshow("lines", img)
cv2.waitKey()
cv2.destroyAllWindows()

#### Circle detection

In [77]:
import cv2
import numpy as np
dog = cv2.imread('dog.jpg')
gray_img = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,120,
param1=100,param2=30,minRadius=0,
maxRadius=0)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(dog,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(dog,(i[0],i[1]),2,(0,0,255),3)
    cv2.imwrite("dog_circles.jpg", dog)
    cv2.imshow("HoughCirlces", dog)

cv2.waitKey()
cv2.destroyAllWindows()

In [80]:
import cv2
import numpy as np
planets = cv2.imread('planets.jpg')
gray_img = cv2.cvtColor(planets, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,120, param1=100,param2=30,minRadius=0, maxRadius=0)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(planets,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(planets,(i[0],i[1]),2,(0,0,255),3)
    cv2.imwrite("planets_circles.jpg", planets)
    cv2.imshow("HoughCirlces", planets)

cv2.waitKey()
cv2.destroyAllWindows()

#### Detecting Shapes

- The detection of shapes with the Hough transform is limited to circles.
- approxPolyDP function:  This function allows the approximation of polygons, so if your
image contains polygons, they will be quite accurately detected, combining the usage
of cv2.findContours and cv2.approxPolyDP